In [1]:
# Importing necessary libraries

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import numpy as np
import matplotlib.pyplot as plt
import os
import time
import random
import glob

from music21 import converter, midi

In [2]:
# Path where the model(weights only) has been stored
checkpoint_path = "./Model_ckpt/model_final_checkpoint"

# Path to dataset file
path_to_file = './Dataset/tunes.txt'

# Path to result directory
path_to_result_dir = './Results'

# RECREATING THE MUSIC GENERATION MODEL

In [3]:
# Read the dataset text file and, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in the file
print('Length of text: {} characters'.format(len(text)))

Length of text: 155224 characters


In [4]:
# Grouping all the unique characters
# 'vocab' is a set containing all unique characters present in 'text'
vocab = sorted(set(text))

print('Number of unique characters is: {}'.format(len(vocab)))

Number of unique characters is: 87


In [5]:
# Creating a mapping table from unique characters to index
char2idx = {ch:i for i, ch in enumerate(vocab)}

# Creating a mapping from index to unique characters
idx2char = np.array(vocab)

# Representing each character in text as its integer representation
text_as_int = np.array([char2idx[ch] for ch in text])

In [6]:
# Length of the vocabulary in chars- or Number of unique characters
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256

In [7]:
# Architecture of the model used for music generation

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size)),
    ])
    
    return model

In [8]:
# Rebuild the model by retrieving the stored weights

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1)

model.load_weights(checkpoint_path)

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            22272     
_________________________________________________________________
gru (GRU)                    (1, None, 256)            394752    
_________________________________________________________________
dropout (Dropout)            (1, None, 256)            0         
_________________________________________________________________
gru_1 (GRU)                  (1, None, 256)            394752    
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 256)            0         
_________________________________________________________________
gru_2 (GRU)                  (1, None, 256)            394752    
_________________________________________________________________
dropout_2 (Dropout)          (1, None, 256)            0

# GENERATE MUSIC

In [9]:
def generate_music_in_ABC(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 900
    
    # Converting the intial string into a seq of indices- vectorizing
    input_seq = [char2idx[ch] for ch in start_string]
    input_seq = tf.expand_dims(input_seq, 0)

    
    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5
    
    # Here batch size == 1
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_seq)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_seq = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))



In [40]:
first_char = 'X'

seq = generate_music_in_ABC(model, start_string=first_char)

music_tunes = []

count = 0

index1 = 0
index2 = 0
new_music_seq = False

for i in range(len(seq)):   
    if not new_music_seq and seq[i:i+2] == 'X:':
        index1 = i
        new_music_seq = True
    
    if new_music_seq and seq[i:i+2] == '\n\n':
        index2 = i
        music_tunes.append(seq[index1:index2])
        new_music_seq = False
        
print('GENERATED MUSICAL TUNES IN ABC NOTATION:')
for music_tune in music_tunes:
    count += 1
    print('\n\nMusical Tune', count, ':\n')
    print(music_tune)
        

GENERATED MUSICAL TUNES IN ABC NOTATION:


Musical Tune 1 :

X: 218
T:Nanth Skelton 1, v 2
% Nottingham Music Database
S:Trad, arr Phil Rowe
M:6/8
K:G
D|"G"G2G "D7"AGF|"G"G2B "D7"G2A|"G"B2B "D7"cBA|"G"B2c "D7"BBc|"G"d2g "D7"dBc|"G"d2g "D7"d2c|"G"BAG "D7"AGF|"G"G3 G2:|
P:B
d/2c/2|"G"Bcd "D7"cBA|"G"GAG G2:|
P:B
d|"G"g2d edB|"D7"ded def|"G"g2d "D7"cBA|"G"Bcd "D7"cBA|"G"GAB "C"cde|"D"d3 -d2:|


Musical Tune 2 :

X: 233
T:Pet
% Nottingham Music Database
S:Trad, arr Phil Rowe
M:6/8
K:D
A|"D"f2e d2c|"G"d2B "D"A2F|"G"G3 "D7"G2A|"G"BAG "D"F3|"G"dBG DGB|dBG d3|"Am"
P:A
|:d|"Em"edB "D"def|"G"g2e "Am"cBA|"D"dfe "G"dcB|"D7"AGF "G"G2::
A|"G"BAB dBd|"C"ece "D7"d2c|"G"Bcd "D7"cBA|"G"BGG G2:|


Musical Tune 3 :

X: 233
T:Pit Daver and His Wife
% Nottingham Music Database
S:Trad, arr Phil Rowe
M:6/8
K:D
A|"D"d2d "A"c2A|"G"BdB "D"AGF|"G"G2e "D"F2d|"Em"E2d "A"cBc|
"D"d2d "A"c2A|"G"BdB "D"AGF|"G"G2e "D"F2d|"Em"E2d "A"cBc|
"D"d2d "A"c2A|"G"BdB "D"AGF|"G"G2e "D"F2d|"Em"E2d "A7"cde|"D"d3 :|


# STORING GENERATED TUNES IN ABC & MIDI FORMATS

In [41]:
# Removing all previously stored midi files in results directory

filelist = glob.glob(os.path.join(path_to_result_dir, '*.abc'))

for file in filelist:
    os.remove(file)
    
filelist = glob.glob(os.path.join(path_to_result_dir, '*.mid'))

for file in filelist:
    os.remove(file)

In [42]:
# Storing the new results

for i, music_tune in enumerate(music_tunes):
    fileName = 'musical_tune_' + str(i+1) 
    
    # Creating an abc file for current music_tune
    abcFilePath = os.path.join(path_to_result_dir, fileName + '.abc')
    abcFile = open(abcFilePath, 'w')
    abcFile.write(music_tune)
    abcFile.close()
    print(abcFilePath, 'created!')
    
    # Converting the abc file into MIDI file using music21
    midiFilePath = os.path.join(path_to_result_dir, fileName + '.mid')
    stream = converter.parse(abcFilePath)
    
    try:
        mf = midi.translate.streamToMidiFile(stream)
        mf.open(midiFilePath, 'wb')
        mf.write()
        mf.close()
        print(midiFilePath, 'created!')
    except:
        print(midiFilePath, 'could not be created! Please use the online abc player link given below to play the file!')
    

./Results/musical_tune_1.abc created!
./Results/musical_tune_1.mid created!
./Results/musical_tune_2.abc created!
./Results/musical_tune_2.mid created!
./Results/musical_tune_3.abc created!
./Results/musical_tune_3.mid created!


=============================================================================== <br>
You can also play the abc files on this site: https://editor.drawthedots.com/ <br>

If an abc music tune takes too long to play on this site, just download the .wav or .midi file and run on your own music player!